In [2]:
import spacy, coreferee
from spacy.matcher import Matcher
import pandas as pd


In [12]:
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 3200000
nlp.add_pipe('coreferee')


In [3]:
import re
def import_file(file_name):
  text = ""


  with open(file_name, encoding="utf8") as f:
    try:
      text = f.readlines()
    except Exception as e:
      print(e)



  string_text = ""
  for line in text:
    string_text += line + "\n"
  string_text = re.sub('[\n]+', '\n', string_text)
  return string_text


In [1]:
filename = "LOTRC"

In [4]:
textString = import_file("./Datasets/"+filename+".txt")

In [10]:
textString[:1000]


'THE LORD\nOF THE RINGS\nChapter 1\nA LONG-EXPECTED PARTY\nWhen Mr. Bilbo Baggins of Bag End announced that he would shortly be celebrating his eleventy-first birthday with a party of special magnificence, there was much talk and excitement in Hobbiton.\nBilbo was very rich and very peculiar, and had been the wonder of the Shire for sixty years, ever since his remarkable disappearance and unexpected return. The riches he had brought back from his travels had now become a local legend, and it was popularly believed, whatever the old folk might say, that the Hill at Bag End was full of tunnels stuffed with treasure. And if that was not enough for fame, there was also his prolonged vigour to marvel at. Time wore on, but it seemed to have little effect on Mr. Baggins. At ninety he was much the same as at fifty. At ninety-nine they began to call him well-preserved; but unchanged would have been nearer the mark. There were some that shook their heads and thought this was too much of a good t

In [11]:
doc = nlp(textString)


In [12]:
import pickle

pickle.dump(doc, open("./Dumps/LOTR/LOTRC_spacy.pkl", "wb"))

In [2]:
import pickle

In [3]:
coreference_resolved = ""
for document in doc:
  res = doc._.coref_chains.resolve(document)
  coreference_resolved += res[0].text +" " if res else document.text + " "


In [4]:
with open("./Datasets/"+filename+"_COREF.txt", "w", encoding="utf8") as f:
  f.write(coreference_resolved)

In [39]:
test = coreference_resolved[:500].split(".")

In [ ]:
test[:200]

In [5]:
textString = import_file("./Datasets/"+filename+"_COREF.txt")


In [24]:
textString[:100]

'\n Chapter 1 \n A LONG - EXPECTED PARTY \n When Mr. Bilbo Baggins of Bag End announced that Baggins wou'

In [15]:
doc = nlp(coreference_resolved)


In [7]:
import pickle

In [8]:
doc = pickle.load(open("./Dumps/LOTR/"+filename+"_COREF_spacy.pkl", "rb"))

In [16]:
import pickle

pickle.dump(doc, open("./Dumps/LOTR/"+filename+"_COREF_spacy.pkl", "wb"))


In [9]:
newDoc = doc

In [10]:
pattern = [
    #First Pattern
    [{"POS": {"IN": ["PROPN"]}, "OP":"*"}, {"POS": {"IN": ["PROPN"]}, "OP":"+"}, 
    {"POS": "VERB"},     {"POS": "PART", "OP": "*"}, 
    {"POS": "ADV", "OP": "*"}, {"POS": {"IN": ["PROPN"]}, "OP": "+"},
    {"POS": {"IN": ["PROPN", "NOUN"]}, "OP":"*"}],
    #Second pattern
    [{"POS": {"IN": ["PROPN"]}, "OP":"*"},
    {"POS": {"IN": ["PROPN"]}, "OP":"+"}, {"POS": "VERB"},
     {"POS": {"IN": ["PART", "ADV", "CONJ"]}, "OP": "*"},  
     {"POS": {"IN": [ "ADV"]}, "OP": "+"},
     {"POS": {"IN": ["PROPN", "NOUN"]}, "OP":"+"}],
    #Third pattern
    [{"POS": {"IN": ["PROPN"]}, "OP":"*"},
    {"POS": {"IN": ["PROPN"]}, "OP":"+"}, {"POS": "VERB"},
     {"POS": {"IN": ["PART", "ADV", "CONJ"]}, "OP": "*"}, 
     {"POS": {"IN": ["ADV"]}, "OP": "+"}, 
     {"POS": {"IN": ["PROPN", ]}, "OP":"+"}, 
     {"POS": {"IN": ["PART", "ADV", "CONJ", "ADJ", "VERB"]}, "OP": "*"},  
     {"POS": {"IN": ["PROPN", "NOUN"]}, "OP":"+"}]
]


Try matching dependency tree

In [13]:
matcher = Matcher(nlp.vocab)
matcher.add("initial",pattern)


In [14]:
matches = matcher(newDoc)

In [15]:
def longest_string_array(array):
    longest_string = ""
    for string in array:
        if len(string) > len(longest_string):
            longest_string = string
    return longest_string

In [16]:
test_df = pd.DataFrame(columns=["Rel","End"])
filteredarray = []
otherArray = []
for match in matches:
  start = match[1]
  end = match[2]
  tempArray = []
  for new_match in matches:
    if new_match[1] == start or new_match[2] == end:
      tempArray.append(newDoc[new_match[1]:new_match[2]])
  if len(tempArray) > 1:
    sentence = longest_string_array(tempArray)

    if sentence.text != "" and sentence not in filteredarray:
      filteredarray.append(sentence)
  else:
    filteredarray.append(newDoc[start:end])
    test_df.loc[len(test_df)] = [newDoc[start:end], newDoc[end].i]


In [17]:
test_df = test_df.sort_values(by="End")

In [18]:
triples = pd.DataFrame(columns=["ENTITY1", "RELATION", "ENTITY2","ADDITIONAL"])
for relation in filteredarray:
  processed_relation = nlp(relation.text)
  i = 0
  entities = [False, False]
  relation = ""
  additional=""
  while i < len(processed_relation):
   
    prev_token = processed_relation[i-1] if i > 0 else None
    token = processed_relation[i]
    entity_name = ""
    if token.pos_ == "PROPN" or token.pos_ == "NOUN":
      
      entity_name += token.text
      j = i+1
      while j <len(processed_relation) and (processed_relation[j].pos_ == "PROPN" or processed_relation[j].pos_ == "NOUN"):
        entity_name += " " + processed_relation[j].text
        j += 1
      i = j-1
      if "subj" in processed_relation[i].dep_ and i < len(processed_relation):
        entities[0] = entity_name
      elif "obj" in processed_relation[i].dep_ and i < len(processed_relation):
        splitted = entity_name.split(" ")

        if len(splitted) > 1 and processed_relation[i].pos_ == "NOUN":
          entity_name = " ".join(splitted[:len(splitted)-1])
          additional = " ".join(splitted[len(splitted)-1:])
    
   
        
        entities[1] = entity_name
      
      
    else:
      relation +=" "+ token.text
    
    i+=1
  if len(entities) == 2 and entities[0] != False and entities[1] != False:
    splitted_second = entities[1].split(" ")
    if len(splitted_second) > 1 and entities[0] == splitted_second[0]:
      if entities[0] != entities[1]:
        triples.loc[len(triples)] = [entities[0], relation,
                                   entities[1],additional]
    else:
      if entities[0] != entities[1]:
        triples.loc[len(triples)] = [entities[0], relation, entities[1],additional]
  

In [20]:
from gensim.models import KeyedVectors
from collections.abc import Mapping
model = KeyedVectors.load('./word2vec-google-news-300.model')
model.vectors[1]


KeyboardInterrupt: 

In [21]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec


In [22]:
import string


def simple_preproc(text):
  """
  see: https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
  """
  return text.translate(str.maketrans('', '', string.punctuation))


In [23]:
with open("./Datasets/"+filename+'_COREF.txt', encoding="utf8") as read_file:
  sentences = [simple_preproc(k).lower().split()
               for k in read_file.readlines()]
model = Word2Vec(sentences=sentences, vector_size=100,
                 window=5, min_count=1, workers=4)


In [24]:
num_errors = 0
triples_vec = pd.DataFrame(columns=["ENTITY1", "RELATION", "ENTITY2","ADDITIONAL","VECTOR"])
for index, row in triples.iterrows():
  try:
    vectors=[]
    for word in row[1].split(" "):
      if word != '':
        vector = model.wv[word.strip()]
        vectors.append(vector)
    triples_vec.loc[len(triples_vec)] = [
        row[0], row[1], row[2], row[3], vectors]
  except Exception as e:
    num_errors +=1

In [25]:
triples_vec.head()

,ENTITY1,RELATION,ENTITY2,ADDITIONAL,VECTOR
0,Bilbo,adopted,Frodo,,"[[-0.04841782, 0.076585054, 0.06452375, -0.039..."
1,Otho,disliked,Bilbo,,"[[-0.014119601, 0.029947247, 0.011427209, -0.0..."
2,Old Odo Proudfoot,removed,Bilbo,feet,"[[-0.1541097, 0.23161198, 0.22655354, -0.06067..."
3,Old Odo Proudfoot,removed,Bilbo,,"[[-0.1541097, 0.23161198, 0.22655354, -0.06067..."
4,Frodo,escorted,Lobelia,,"[[-0.012109284, 0.01570673, 0.016003642, 0.002..."


In [26]:
import numpy as np
temp_dataset = triples_vec
for index, row in triples_vec.iterrows():
  vectors = row[4]
  if len(vectors) > 1:
    average = np.mean(vectors, axis=0)
    temp_dataset.loc[index]["VECTOR"] = average
    
  else:
    temp_dataset.loc[index]["VECTOR"] = row[4][0]
temp_dataset.head(20)

,ENTITY1,RELATION,ENTITY2,ADDITIONAL,VECTOR
0,Bilbo,adopted,Frodo,,"[-0.04841782, 0.076585054, 0.06452375, -0.0392..."
1,Otho,disliked,Bilbo,,"[-0.014119601, 0.029947247, 0.011427209, -0.00..."
2,Old Odo Proudfoot,removed,Bilbo,feet,"[-0.1541097, 0.23161198, 0.22655354, -0.060672..."
3,Old Odo Proudfoot,removed,Bilbo,,"[-0.1541097, 0.23161198, 0.22655354, -0.060672..."
4,Frodo,escorted,Lobelia,,"[-0.012109284, 0.01570673, 0.016003642, 0.0024..."
5,Frodo,saw,Gandalf,,"[0.08047636, -0.36959633, 0.8828216, 0.4029530..."
6,Gandalf,paid,Frodo,,"[-0.01753119, 0.15533338, 0.044456325, -0.0911..."
7,Frodo,welcomed,Sam,,"[-0.04632163, 0.09652247, 0.057476882, -0.0269..."
8,Sam,pressed,Gandalf,,"[-0.05358205, 0.13839875, 0.13058482, 0.041677..."
9,Orcs,saw,Isildur,,"[0.08047636, -0.36959633, 0.8828216, 0.4029530..."


In [27]:
from sklearn.cluster import DBSCAN  # To instantiate and fit the model
from sklearn.metrics import pairwise_distances  # For Model evaluation
from sklearn.neighbors import NearestNeighbors  # For Hyperparameter Tuning
from sklearn.preprocessing import StandardScaler


In [28]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.05, min_samples=2).fit(temp_dataset["VECTOR"].values.tolist())  # fitting the model


In [29]:
unique = set(dbscan.labels_)

In [30]:
temp_dataset["LABEL"] = None

In [31]:
for index, row in temp_dataset.iterrows():
  temp_dataset.loc[index,"LABEL"] = dbscan.labels_[index]

In [32]:
import random

for label in unique:
  current_label_items = temp_dataset.loc[temp_dataset["LABEL"] == label]
  relations = current_label_items["RELATION"].values
  average = np.mean(current_label_items["VECTOR"].values, axis=0)
  new_selected_relation = model.wv.most_similar(average)
  # print(model.wv.most_similar(average)[0][0])
  temp_dataset.loc[temp_dataset["LABEL"] == label,
                   "NEW_RELATION"] = new_selected_relation[0][0]
  


In [33]:
temp_dataset.to_csv("./Datasets/"+filename+"_TRIPLES.csv")
